# Gold Notebook
### This notebook aggregates and combines silver tables to useful tables for downstream consumers to use to answer business logic questions.

In [1]:
## This time dimension table can be very useful for Vizualization

spark.sql("""
CREATE OR REPLACE TABLE z_gold_dim_date USING DELTA AS
WITH dates AS (
  SELECT explode(sequence(to_date('1993-01-01'),to_date('2000-12-31'),interval 1 day)) AS full_date
)
SELECT
  cast(date_format(full_date,'yyyyMMdd') AS int) AS date_sk,
  full_date AS full_date,
  year(full_date) AS year,
  quarter(full_date) AS quarter,
  month(full_date) AS month,
  dayofmonth(full_date) AS day_of_month,
  dayofweek(full_date) AS day_of_week,     -- 1=Sunday
  date_format(full_date,'EEEE') AS day_name,
  date_format(full_date,'MMMM') AS month_name,
  concat(year(full_date),lpad(month(full_date),2,'0')) AS year_month,
  dayofweek(full_date) IN (1,7) AS is_weekend
FROM dates
""")

display(spark.sql("SELECT * FROM z_gold_dim_date LIMIT 12"))
print(f"Total rows: {spark.table('z_gold_dim_date').count():,}")

StatementMeta(, 4f221280-4dca-4deb-8d3d-3af13f9ed374, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 85192df5-6c6e-491a-b7ca-5799626ee229)

Total rows: 2,922


In [2]:
## gold_dim_client has relevant information of clients wrt to loans, card and user data

spark.sql("""
CREATE OR REPLACE TABLE z_gold_dim_client USING DELTA AS
WITH card_ownership AS (
  SELECT disp.client_id,
         COUNT(card.card_id) AS cards_issued,
         MAX(card.issued_date) AS latest_card_date
  FROM silver_dim_disposition disp
  LEFT JOIN silver_dim_card card ON disp.disp_id = card.disp_id
  GROUP BY disp.client_id
),
loan_flag AS (
  SELECT disp.client_id,
         COUNT(loan.loan_id) AS loan_count,
         MAX(CASE WHEN loan.is_default = 1 THEN 1 ELSE 0 END) AS ever_defaulted
  FROM silver_dim_disposition disp
  LEFT JOIN silver_fact_loan loan ON disp.account_id = loan.account_id
  GROUP BY disp.client_id
)
SELECT
  c.client_id,
  c.district_id,
  d.district_name,
  d.region,
  c.gender,
  c.birth_date,
  c.age_1999,
  c.age_1999 + (YEAR(current_date()) - 1999) AS age_today,
  COALESCE(card_ownership.cards_issued,0)>0 AS has_credit_card,
  COALESCE(card_ownership.cards_issued,0) AS credit_cards_count,
  card_ownership.latest_card_date AS latest_card_issued_date,
  COALESCE(loan_flag.loan_count,0)>0 AS has_loan,
  COALESCE(loan_flag.loan_count,0) AS loan_count,
  COALESCE(loan_flag.ever_defaulted,0)=1 AS ever_defaulted,
  d.unemployment_1996 AS unemployment_rate,
  d.avg_salary AS avg_salary_district,
  d.inhabitants AS district_population,
  d.urban_ratio AS urban_ratio
FROM silver_dim_client c
LEFT JOIN silver_dim_district d ON c.district_id = d.district_id
LEFT JOIN card_ownership ON c.client_id = card_ownership.client_id
LEFT JOIN loan_flag ON c.client_id = loan_flag.client_id
""")

display(spark.sql("SELECT * FROM z_gold_dim_client LIMIT 10"))
print(f"Clients in gold: {spark.table('z_gold_dim_client').count():,}")

StatementMeta(, 4f221280-4dca-4deb-8d3d-3af13f9ed374, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3073c7ff-234e-403e-98a4-7f35f2361e1a)

Clients in gold: 5,369


In [3]:
## This is a more robust gold table combining the full 360 client profile (lifetime spend, latest card, defaulter flag, etc.)

spark.sql("""
CREATE OR REPLACE TABLE z_gold_client_360 USING DELTA AS
SELECT
  cli.client_id,
  cli.gender,
  cli.age_1999,
  cli.district_name,
  cli.region,
  cli.avg_salary_district,
  cli.unemployment_rate,

  -- Card
  cli.has_credit_card,
  cli.credit_cards_count,
  latest_card.card_type AS latest_card_type,
  latest_card.latest_issued_date AS latest_card_issued,
  cli.age_1999 + (YEAR(latest_card.latest_issued_date) - 1999) AS age_when_latest_card_issued,

  -- Loan (owner only)
  loan.client_id IS NOT NULL AS has_loan,
  loan.loan_status,
  loan.amount AS loan_amount_czk,
  loan.monthly_payment,
  loan.is_default,

  -- Lifetime spending (1993–1999 only)
  COALESCE(spend.total_spent_czk,0) AS lifetime_total_spend_czk,
  COALESCE(spend.card_spend_czk,0) AS lifetime_card_spend_czk,
  COALESCE(spend.trans_count,0) AS lifetime_trans_count,
  COALESCE(spend.card_trans_count,0) AS lifetime_card_trans_count,
  ROUND(COALESCE(spend.avg_ticket_czk,0),0) AS avg_transaction_czk,

  -- Accurate 1999-context flags
  cli.age_1999 < 30 AND cli.has_credit_card AS is_young_card_holder,
  cli.age_1999 >= 60 AS is_senior_client,
  COALESCE(loan.is_default,false) AS is_loan_defaulter,
  COALESCE(spend.card_spend_czk,0)>500000 AS is_high_value_card_user,
  cli.avg_salary_district>10000 AND cli.has_credit_card AS is_premium_segment,
  cli.has_credit_card AND COALESCE(spend.card_trans_count,0)=0 AS card_never_used

FROM z_gold_dim_client cli
LEFT JOIN (
  SELECT disp.client_id,
         ANY_VALUE(card.card_type) AS card_type,
         MAX(card.issued_date) AS latest_issued_date
  FROM silver_dim_card card
  JOIN silver_dim_disposition disp ON card.disp_id = disp.disp_id
  GROUP BY disp.client_id
) latest_card ON cli.client_id = latest_card.client_id
LEFT JOIN (
  SELECT disp.client_id,
         ANY_VALUE(loan.loan_status) AS loan_status,
         ANY_VALUE(loan.amount) AS amount,
         ANY_VALUE(loan.monthly_payment) AS monthly_payment,
         ANY_VALUE(loan.is_default) AS is_default
  FROM silver_fact_loan loan
  JOIN silver_dim_disposition disp 
    ON loan.account_id = disp.account_id AND disp.is_owner = true
  GROUP BY disp.client_id
) loan ON cli.client_id = loan.client_id
LEFT JOIN (
  SELECT disp.client_id,
         SUM(t.amount) AS total_spent_czk,
         SUM(CASE WHEN t.operation='Credit Card Withdrawal' THEN t.amount ELSE 0 END) AS card_spend_czk,
         COUNT(*) AS trans_count,
         COUNT(CASE WHEN t.operation='Credit Card Withdrawal' THEN 1 END) AS card_trans_count,
         AVG(t.amount) AS avg_ticket_czk
  FROM silver_fact_transaction t
  JOIN silver_dim_disposition disp ON t.account_id = disp.account_id
  GROUP BY disp.client_id
) spend ON cli.client_id = spend.client_id
""")

print(f"Clients in z_gold_client_360: {spark.table('z_gold_client_360').count():,}")
display(spark.sql("SELECT * FROM z_gold_dim_client LIMIT 10"))

StatementMeta(, 4f221280-4dca-4deb-8d3d-3af13f9ed374, 5, Finished, Available, Finished)

Clients in z_gold_client_360: 5,369


SynapseWidget(Synapse.DataFrame, f10e12b8-a878-4739-8a30-5ad57857c183)

In [4]:
# Full enriched transaction fact with client_id, card_type, is_card_transaction, gold card flag, etc.

spark.sql("""
CREATE OR REPLACE TABLE z_gold_fact_transaction USING DELTA AS
SELECT
  t.trans_id,
  t.account_id,
  t.date AS trans_date,
  YEAR(t.date) AS trans_year,
  MONTH(t.date) AS trans_month,
  t.type AS trans_type,           -- Credit / Debit
  t.operation,                     -- Credit Card Withdrawal, Withdrawal in Cash, etc.
  t.amount AS amount_czk,
  t.balance AS balance_after_czk,
  t.k_symbol AS k_symbol_en,
  t.partner_bank,
  t.partner_account,

  -- Client & Account context
  cli.client_id,
  cli.gender,
  cli.district_name,
  cli.region,
  cli.avg_salary_district,
  cli.unemployment_rate,
  cli.has_credit_card,
  cli.has_loan,
  cli.ever_defaulted,

  -- Card context (only when applicable)
  card.card_id,
  LOWER(card.card_type) AS card_type,           -- classic / junior / gold / null
  card.issued_date AS card_issued_date,

  -- Flags — instant filtering power
  (t.operation = 'Credit Card Withdrawal') AS is_card_transaction,
  (LOWER(card.card_type) = 'gold') AS is_gold_card,
  (LOWER(card.card_type) = 'junior') AS is_junior_card,
  (cli.has_loan = true AND cli.ever_defaulted = true) AS is_loan_defaulter,
  (t.type = 'Debit') AS is_withdrawal,
  (t.type = 'Credit') AS is_credit

FROM silver_fact_transaction t
JOIN silver_dim_disposition disp ON t.account_id = disp.account_id
JOIN z_gold_dim_client cli ON disp.client_id = cli.client_id
LEFT JOIN silver_dim_card card 
  ON disp.disp_id = card.disp_id 
 AND t.date >= card.issued_date
""")

print(f"Clients in z_gold_fact_transaction: {spark.table('z_gold_fact_transaction').count():,}")
display(spark.sql("SELECT * FROM z_gold_fact_transaction LIMIT 10"))

StatementMeta(, 4f221280-4dca-4deb-8d3d-3af13f9ed374, 6, Finished, Available, Finished)

Clients in z_gold_fact_transaction: 1,262,625


SynapseWidget(Synapse.DataFrame, 3a4a5b1b-fcc7-46f8-b1ad-ce2a2b99a731)

In [5]:
## Monthly credit card spending KPIs by client and card type (total spent, avg ticket, etc.)

spark.sql("""
CREATE OR REPLACE TABLE z_gold_monthly_card_spending USING DELTA AS
SELECT
  YEAR(trans_date) AS year,
  MONTH(trans_date) AS month,
  CONCAT(YEAR(trans_date), '-', LPAD(MONTH(trans_date),2,'0')) AS year_month,

  client_id,
  gender,
  district_name,
  region,
  card_type,

  COUNT(*) AS trans_count,
  ROUND(SUM(amount_czk),0) AS total_spent_czk,
  ROUND(AVG(amount_czk),0) AS avg_ticket_czk,
  ROUND(MAX(amount_czk),0) AS max_ticket_czk,

  (is_gold_card = 1) AS is_gold_card,
  has_loan AS client_has_loan,
  ever_defaulted AS client_ever_defaulted,
  ROUND(avg_salary_district,0) AS avg_salary_district,
  ROUND(unemployment_rate,2) AS unemployment_rate

FROM z_gold_fact_transaction
WHERE is_card_transaction = 1
GROUP BY ALL
""")

print(f"Clients in z_gold_monthly_card_spending: {spark.table('z_gold_monthly_card_spending').count():,}")
display(spark.sql("SELECT * FROM z_gold_monthly_card_spending LIMIT 10"))

StatementMeta(, 4f221280-4dca-4deb-8d3d-3af13f9ed374, 7, Finished, Available, Finished)

Clients in z_gold_monthly_card_spending: 5,597


SynapseWidget(Synapse.DataFrame, 41f07493-9ac9-4de4-a901-ddc67c9e76be)